In [ ]:
import os
from dotenv import load_dotenv
from datasets import load_dataset
from smolagents import CodeAgent, InferenceClientModel ###LiteLLMModel, HfApiModel
import yaml
from tools.final_answer import FinalAnswerTool
from tools.math_operations import MathOperationsTool
from tools.visit_webpage import VisitWebpageTool
from tools.web_search import DuckDuckGoSearchTool
from tools.wikipedia_search import WikipediaSearchTool
from tools.arxiv_search import ArxivSearchTool
from tools.rag_search import RAGSearchTool
from tools.code_execution import CodeExecutionTool
from tools.document_processing import DocumentProcessingTool
from tools.image_processing import ImageProcessingTool
from tools.web_scraping import WebScrapingTool
from tools.youtube_processing import YouTubeVideoProcessorTool

In [ ]:
visit_webpage = VisitWebpageTool()
web_search = DuckDuckGoSearchTool(max_results=5)
math_tools = MathOperationsTool()
final_answer = FinalAnswerTool()
wikipedia_search = WikipediaSearchTool(load_max_docs=2)
arxiv_search = ArxivSearchTool(load_max_docs=3)
rag_search = RAGSearchTool(persist_dir="rag_db")
code_execution = CodeExecutionTool()
document_processing = DocumentProcessingTool(temp_dir="doc_processing")
image_generation_tool = ImageProcessingTool()
web_scraping = WebScrapingTool()
youtube_processing = YouTubeVideoProcessorTool()

In [ ]:
customs_tools = [    
    visit_webpage, 
    web_search, 
    math_tools,
    wikipedia_search, 
    arxiv_search,
    rag_search,
    web_scraping,
    code_execution, 
    document_processing, 
    image_generation_tool,
    final_answer
]

In [ ]:
# Inicialize HuggingFace agent
load_dotenv()
# Get the Hugging Face API key from the environment
api_key = os.getenv("HUGGINGFACE_API_KEY")
# Check that the key was loaded
if not api_key:
    raise ValueError("HUGGINGFACE_API_KEY not found. Please check your .env file.")
HFModel = InferenceClientModel(
    max_tokens=2096,
    temperature=0,
    model_id='deepseek-ai/DeepSeek-R1-Distill-Qwen-32B',
    provider="hf-inference",
    api_key=api_key
)


In [ ]:
# Cargar las plantillas de los prompts
with open("prompt.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)

In [ ]:
 #3. Inicializar modelo Ollama local
from smolagents import LiteLLMModel
local_model = LiteLLMModel(
    model_id="ollama/qwen2.5-coder:7b",    
    #model_id="ollama/gemma3:1b",    
    api_base="http://localhost:11434",
    temperature=0,
    max_tokens=2096,
    request_timeout=900,
)

In [ ]:
# 4. Crear agente con el modelo local
agent = CodeAgent(
    model=local_model,
    #model = HFModel,
    tools = customs_tools, 
    #additional_authorized_imports = additional_imports,
    #prompt_templates=prompt_templates,    
    verbosity_level = 2,
    max_steps=20,
)

In [ ]:
print(agent.prompt_templates["system_prompt"])


In [ ]:
dataset_validation = load_dataset(
    "gaia-benchmark/GAIA",
    '2023_all',    
    split="validation",
    token=os.getenv("HF_TOKEN"),    
    trust_remote_code=True,
)

In [ ]:
#dataset_test = load_dataset(
#    "gaia-benchmark/GAIA",
#    '2023_all',    
#    split="test",
#    token=os.getenv("HF_TOKEN"),
#    trust_remote_code=True,
#)

In [ ]:
print("---- Test set ----")
indices = list(range(1))
split_dataset_validation = dataset_validation.select(indices)
for example in split_dataset_validation:
    print(example)  

In [ ]:
# 5. Ejecutar preguntas y recoger respuestas
predictions = []
for example in split_dataset_validation:
    q = example["Question"]
    print("Agent started running...")
    pred = agent.run(q)
    predictions.append(str(pred).strip())
    print("Agent finished running...")
    print(f"QUESTION: {q}\nANSWER: {pred}\n")

In [ ]:
# 6. Calcular exact‑match accuracy
gold = [ex["Final answer"].strip() for ex in split_dataset_validation]
correct = sum(1 for p, g in zip(predictions, gold) if p == g)
acc = correct / len(gold)
print(f"Exact‑match accuracy on GAIA dev: {acc:.2%}")


************ LOCAL TEST **************


In [ ]:
q = "According to the World Bank, which countries had gross savings of over 35% of GDP for every year in the period 2001-2010? Give your answer as a comma-separated list of countries in alphabetical order. Use the countries most common names in english when answering."
pred = agent.run(q)
print(f"QUESTION: {q}\nANSWER: {pred}\n")


In [ ]:
wikipedia_info = wikipedia_search(query="Mercedes Sosa")
print(wikipedia_info)

In [ ]:
# Testing the tools
# arxiv_search
papers = arxiv_search(query="AI regulation submission date:2022-06")
print("arxiv_search sucess")

# rag_search
rag_result = rag_search(
    action="add",
    content=papers,
    metadata={"topic": "transformers", "subtopic": "time_series", "year": "2023"}
)
print("rag_search sucess")

# document_processing
doc_analysis = document_processing(
    action="analyze_pdf",
    content="financial_report.pdf",
    query="What are the total revenue figures for 2021 and 2022?"
)
print("document_processing sucess")

# math_tools
growth = math_tools(
    operation="subtract",
    a=1.2,
    b=1.5
)
print("math_tools sucess")

# internet_search
search_results = web_search(query="latest deep learning frameworks comparison 2023")
print("internet_search sucess")

# web_scraping
scrapping_content = web_scraping(
    url="https://www.datacamp.com/tutorial/how-transformers-work",
    action="element",
    selector="table.comparison",
    attributes=["class"]
)
print("web_scraping sucess")

# wikipedia_search
wikipedia_info = wikipedia_search(query="Mercedes Sosa")
print("wikipedia_search sucess")

# code_execution
code = '''
def divide_numbers(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b
    
# Test the function
print(divide_numbers(10, 2))
print(divide_numbers(5, 5))
'''
result = code_execution(code=code)
print("code_execution sucess")

# document_processing
text = document_processing(
    action="ocr",
    content="presentation.jpg"
)
print("document_processing sucess")

text = final_answer("Based on the financial report, the total revenues for 2021 and 2022 is 1.2 and 1.5 respectively.")
print("final_answer sucess")

In [ ]:
#url = "https://en.wikipedia.org/wiki/List_of_secretaries_of_Homeland_Security"                                     
url = "https://data.worldbank.org/indicator/NY.GDS.TOTL.ZS"
web_content = visit_webpage.forward(url=url)                                                                             
print(web_content)


In [ ]:
query = "List of United States secretaries of homeland security before April 2019"                               
search_results = web_search.forward(query=query)                                                                 
print(search_results)  # Use print to output important information for the next step.       